In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "3호모델의모델"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'3호모델의모델_Y_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(list_data.copy(),
                                  plus_one="BW금액"
                                 ,plus_two="CB금액", return_origianl=False)

#
concat_together1["2012-01"].columns

Index(['BW금액+CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '유상증자금액',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="BW금액+CB금액"
                                 ,plus_two="유상증자금액", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['BW금액+CB금액+유상증자금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [12]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="타법인주식및출자증권취득결정금액"
                                 ,plus_two="특수관계인에대한자금대여금액", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [13]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="상호변경카운트"
                                 ,plus_two="업종변경카운트", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['상호변경카운트+업종변경카운트', '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [14]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송"
                                 ,plus_two="불성실공시법인지정카운트", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권',
       '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비',
       '최대주주비율_변동성'],
      dtype='object')

In [15]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="경영권분쟁소송+불성실공시법인지정카운트"
                                 ,plus_two="조회공시요구카운트", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [16]:
drop_togehter = tools.dict_data_drop(concat_together7.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)
scaled_together["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [17]:
# 최초 모델을 돌리고
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())

100%|██████████| 100/100 [00:07<00:00, 12.72it/s]


In [18]:
# 최초모델의 결과
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict,
                                                            raw_data = scaled_together, 
                                                            label_data=label_data_pp_man, freq=freq)

# 모델의 모델(나쁜놈들도 다시한번 굴림)
model2_save_dict, filtered_data = evaluate.cluster_of_cluster_model(model_save=model_save_dict,
                                  raw_data=scaled_together,
                                  eval_label=eval_one, num_k=3, max_sample=100)

100%|██████████| 100/100 [00:01<00:00, 56.96it/s]


### 모델의 모델 평가

In [19]:
size =evaluate.eval_cluster_size(model2_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

302.0
127.0


,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,165.0,129.0,226.0,174.0,198.0,179.0,175.0,154.0,251.0,216.0
1,194.0,175.0,127.0,151.0,198.0,204.0,195.0,166.0,245.0,302.0
2,165.0,197.0,183.0,201.0,140.0,185.0,227.0,255.0,196.0,227.0
기업수,524.0,501.0,536.0,526.0,536.0,568.0,597.0,575.0,692.0,745.0


In [20]:
eval, eval_original = evaluate.eval_cluster_result(model2_save_dict, raw_data = filtered_data,
                                                    label_data=label_data_pp_man, freq=freq)

eval.to_csv("{}_관리종목.csv".format(model_name))
eval_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [21]:
eval_t,eval_t_ori = evaluate.eval_cluster_result(model2_save_dict, raw_data = filtered_data, 
                                                 label_data=label_data_pp_war, freq=freq)

eval_t.to_csv("{}_투자환기종목.csv".format(model_name))
eval_t_ori.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [22]:
print(evaluate.calculate_catch_probability(eval, eval_t, None))
print(evaluate.calculate_match_probability(eval, eval_t,None))
evaluate.calculate_catch_probability(eval_original, eval_t_ori, None)

0.6255262683165834
1.0


0.06527753528299142

In [23]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval, filtered_data, model2_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

BW금액+CB금액+유상증자금액                          0.167111
최대주주비율_변동성                                0.161925
차입금의존도                                    0.148560
결산기간_거래량증가                                0.115814
최대주주비율                                    0.076461
경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트            0.071152
최대주주변경카운트                                 0.049362
당기순이익                                     0.035480
판매 및 관리비                                  0.033876
타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액           0.032022
매출채권                                      0.027339
결산기간_주가변동률                                0.019143
최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액    0.016928
영업이익                                      0.016356
영업현금흐름                                    0.008546
상호변경카운트+업종변경카운트                           0.008446
총자본                                       0.007995
매출총이익                                     0.003485
dtype: float64

고위험 클러스터의 평균 재무, 공시 상황 출력

* eval_cluster Result의 결과를 받는다 / model_save_dict / raw_data -> drop_together (스케일 전 데이터를 받아야함)

In [24]:
# scaled data를 토대로 원본 데이터를 추출한다
key = filtered_data.keys()
original_data_dict = {}
for t in key:
    idx = filtered_data[t].index
    origianl_data = drop_togehter[t]
    original_data_dict[t] = origianl_data.loc[idx]

In [25]:
dict = evaluate.see_cluster_info(model2_save_dict, eval, original_data_dict)
data = pd.DataFrame()
col = dict.keys()

for t, df in dict.items():
    data = pd.concat([data, df], axis=1)

data.columns = col
data.T.to_csv("{}_평균정보.csv".format(model_name))
data.T

,경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트,상호변경카운트+업종변경카운트,타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액,최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액,BW금액+CB금액+유상증자금액,최대주주변경카운트,결산기간_거래량증가,결산기간_주가변동률,당기순이익,매출채권,매출총이익,영업이익,영업현금흐름,차입금의존도,총자본,최대주주비율,판매 및 관리비,최대주주비율_변동성
2012-01,0.876289,0.190722,3.396197e+09,1.124586e+09,9.142537e+09,0.226804,1.534042,1.124604,-2.829207e+06,2.011001e+07,1.189077e+06,-1.775450e+06,-7.842252e+05,35.779008,3.669841e+07,23.353527,2.964527e+06,-0.007204
2013-01,1.170543,0.100775,7.044331e+09,6.941358e+08,1.121056e+10,0.286822,1.193008,4.045331,-2.192167e+07,1.030605e+08,-5.907294e+06,-1.354008e+07,-5.944422e+06,45.390775,8.460478e+07,26.446231,7.634051e+06,-0.009146
2014-01,0.830601,0.136612,2.174264e+09,5.507650e+08,1.702979e+10,0.295082,1.154083,2.934415,-1.505111e+07,3.207541e+07,-4.054363e+06,-7.276713e+06,-3.283195e+06,41.724850,2.110917e+07,23.089392,3.222349e+06,0.078811
2015-01,1.377483,0.317881,3.815461e+09,4.435836e+09,1.064353e+10,0.629139,2.280547,1.473855,-3.845777e+06,1.543932e+07,7.478290e+05,-1.932547e+06,-1.232882e+06,33.010248,2.863586e+07,22.850466,2.680376e+06,inf
2016-01,2.035714,0.264286,5.741891e+09,8.763806e+09,3.295459e+10,0.885714,inf,1.493540,-9.794248e+06,2.126723e+07,-4.194118e+05,-4.613312e+06,-1.108859e+06,34.132054,3.678496e+07,19.214785,4.193900e+06,-0.021284
2017-01,1.156757,0.297297,4.344069e+09,7.710118e+09,2.452910e+10,0.627027,inf,1.114593,-2.128745e+07,2.230294e+07,7.940046e+05,-3.442349e+06,1.314472e+05,32.598054,3.507679e+07,18.928410,4.236354e+06,inf
2018-01,1.302857,0.171429,8.653020e+09,9.833360e+09,3.426908e+10,0.640000,inf,1.632192,-4.766777e+06,2.116746e+07,1.590029e+06,-2.605054e+06,-1.917383e+06,34.039171,6.407607e+07,20.700788,4.195083e+06,0.004187
2019-01,1.402597,0.266234,8.762472e+09,8.687004e+09,2.437774e+10,0.701299,1.687756,0.921384,-5.109246e+06,1.383163e+07,1.986962e+06,-1.936473e+06,-1.487677e+06,40.212938,4.719275e+07,18.459545,3.923427e+06,inf
2020-01,0.987755,0.212245,8.388963e+09,4.062707e+09,2.464784e+10,0.334694,inf,0.572970,-9.167946e+06,1.629909e+07,3.658235e+05,-5.010418e+06,-2.569837e+06,40.234296,7.925670e+07,21.102510,5.372435e+06,inf
2021-01,0.463576,0.115894,3.402765e+09,3.771800e+09,2.324853e+10,0.215232,1.263799,0.668342,-4.520961e+06,1.387112e+07,2.645396e+06,-2.439101e+06,-1.814622e+06,29.655215,5.961814e+07,27.449008,5.079986e+06,-0.015105


In [26]:
data.T.mean().to_csv("{}_평균정보의평균.csv".format(model_name))

In [27]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(filtered_data["2020-01"],
                                        model2_save_dict["2020-01"]["model"],
                                         answer_data=label_data_pp_man["2020-01"],
                                         fig_title="{}_관리종목".format(model_name))

In [28]:
MyKmeans.visualize_3d_with_answer(filtered_data["2020-01"],
                                  model2_save_dict["2020-01"]["model"],
                                 answer_data=label_data_pp_war["2020-01"],
                                 fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000040,0.287996,0.070184,-0.332410,1,2.5
A000440,-0.562481,0.166796,-0.240115,0,2.5
A000650,-0.457632,-0.113229,0.134509,2,2.5
A000890,0.240952,0.177628,-0.291768,1,2.5
A000950,0.443058,-0.324850,0.121259,1,2.5
...,...,...,...,...,...
A317850,-0.015477,-0.364284,-0.195992,2,2.5
A317870,-0.378931,0.060924,-0.456732,0,2.5
A318000,-0.814776,0.385065,-0.087940,0,2.5
A322510,0.036659,-0.299957,0.128157,2,2.5
